### Image Classification dataset preparation

### FIXME

1. Choose between default and custom dataset in FIXME 1
1. Assign path of DATA_DIR in FIXME 2
1. Assign Cloud credentials in FIXME 3

### Example dataset source and structure <a class="anchor" id="head-1.1"></a>

**If using custom dataset; it should follow this dataset structure, and skip running** "**Split dataset into train and val sets**"
```
DATA_DIR
├── classes.txt
├── classmap.json
├── images_test
│   ├── class_name_1
│   │   ├── image_name_1.jpg
│   │   ├── image_name_2.jpg
│   │   ├── ...
|   |   ... 
│   └── class_name_n
│       ├── image_name_3.jpg
│       ├── image_name_4.jpg
│       ├── ...
├── images_train
│   ├── class_name_1
│   │   ├── image_name_5.jpg
│   │   ├── image_name_6.jpg
|   |   ...
│   └── class_name_n
│       ├── image_name_7.jpg
│       ├── image_name_8.jpg
│       ├── ...
|
└── images_val
    ├── class_name_1
    │   ├── image_name_9.jpg
    │   ├── image_name_10.jpg
    │   ├── ...
    |   ...
    └── class_name_n
        ├── image_name_11.jpg
        ├── image_name_12.jpg
        ├── ...
```
- Each class name folder should contain the images corresponding to that class
- Same class name folders should be present across images_test, images_train and images_val
- classes.txt is a file which contains the names of all classes (each name in a separate line)
- classes.json is a json file where the key is classname value is integer, for VOC: {"aeroplane": 0, "bicycle": 1, "bird": 2, "boat": 3, "bottle": 4, "bus": 5, "car": 6, "cat": 7, "chair": 8, "cow": 9, "diningtable": 10, "dog": 11, "horse": 12, "motorbike": 13, "person": 14, "pottedplant": 15, "sheep": 16, "sofa": 17, "train": 18, "tvmonitor": 19}

In [ ]:
import os

dataset_to_be_used = "default" #FIXME1 #default/custom; default for the dataset used in this tutorial notebook; custom for a different dataset
DATA_DIR = "/data/" #FIXME2
os.environ['DATA_DIR']= DATA_DIR
!mkdir -p $DATA_DIR

### Dataset download and pre-processing <a class="anchor" id="head-1"></a>

In [ ]:
if dataset_to_be_used == "default":
    assert os.path.exists(os.path.join(DATA_DIR,"VOCtrainval_11-May-2012.tar"))
    !tar -xf $DATA_DIR/VOCtrainval_11-May-2012.tar -C $DATA_DIR
    assert (os.path.exists(f"{DATA_DIR}/VOCdevkit/"))
    !rm -rf $DATA_DIR/split

#### Split dataset into train and val sets

In [ ]:
# Split dataset into train and val sets
!python3 -m pip install numpy pandas==1.5.1 tqdm
!python3 classification/dataset_split.py
assert (os.path.exists(f"{DATA_DIR}/split/images_train/"))
assert (os.path.exists(f"{DATA_DIR}/split/images_val/"))
assert (os.path.exists(f"{DATA_DIR}/split/images_test/"))

### Create Tar files to upload

In [ ]:
!mkdir -p $DATA_DIR/cloud_folders/data/classification_train $DATA_DIR/cloud_folders/data/classification_val $DATA_DIR/cloud_folders/data/classification_test

!tar -C $DATA_DIR/split/ -czf $DATA_DIR/cloud_folders/data/classification_train/images_train.tar.gz images_train
!tar -C $DATA_DIR/split/ -czf $DATA_DIR/cloud_folders/data/classification_val/images_val.tar.gz images_val
!tar -C $DATA_DIR/split/ -czf $DATA_DIR/cloud_folders/data/classification_test/images_test.tar.gz images_test

!cp $DATA_DIR/split/classes.txt $DATA_DIR/split/classmap.json $DATA_DIR/cloud_folders/data/classification_train/
!cp $DATA_DIR/split/classes.txt $DATA_DIR/split/classmap.json $DATA_DIR/cloud_folders/data/classification_val/
!cp $DATA_DIR/split/classes.txt $DATA_DIR/split/classmap.json $DATA_DIR/cloud_folders/data/classification_test/

### Final step: Upload the /data folder to your cloud storage and move on to running the API requests example notebooks
When you do a ls of your bucket it should have /data folder and the subfolders we created above within in (classification_train, classification_val, classification_test)

In [ ]:
!python3 -m pip install --upgrade awscli
ACCESS_KEY=FIXME3.1
SECRET_KEY=FIXME3.2
BUCKET_NAME=FIXME3.3
!AWS_ACCESS_KEY_ID={ACCESS_KEY} AWS_SECRET_ACCESS_KEY={SECRET_KEY} aws s3 cp {DATA_DIR}/cloud_folders/data/classification_train s3://{BUCKET_NAME}/data/classification_train --recursive
!AWS_ACCESS_KEY_ID={ACCESS_KEY} AWS_SECRET_ACCESS_KEY={SECRET_KEY} aws s3 cp {DATA_DIR}/cloud_folders/data/classification_val s3://{BUCKET_NAME}/data/classification_val --recursive
!AWS_ACCESS_KEY_ID={ACCESS_KEY} AWS_SECRET_ACCESS_KEY={SECRET_KEY} aws s3 cp {DATA_DIR}/cloud_folders/data/classification_test s3://{BUCKET_NAME}/data/classification_test --recursive

In [ ]:
# This will be the paths in your API/TAO-CLIENT Notebooks
train_dataset_path =  "/data/classification_train"
eval_dataset_path = "/data/classification_val"
test_dataset_path = "/data/classification_test"